## Importing Libraries

In [18]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import cloudscraper
import warnings
warnings.filterwarnings("ignore")

## Retrieving data from Sitemap

In [19]:
base_url = 'https://www.ghanaweb.com'

In [20]:
URL = "https://www.ghanaweb.com/GhanaHomePage/sitemap.php"
page = requests.get(URL)

#print(page.text)

In [21]:
soup = BeautifulSoup(page.content, 'html.parser')

In [22]:
results = soup.find(id = "medsection2")

In [23]:
site_map_data = results.find_all("dl", class_ = "resources")

In [24]:
df_columns = ['Section','url']
sections_df = pd.DataFrame(columns = df_columns)

In [25]:
for sections in site_map_data:
    urls = sections.find_all("a")
    for url in urls:
        temp_row = pd.DataFrame(data = [[url.text,base_url+url.get('href')]], columns = df_columns)
        sections_df = pd.concat([sections_df,temp_row], ignore_index=True)

In [26]:
sections_df.head()

,Section,url
0,"Abroad, Ghanaians",https://www.ghanaweb.com/GhanaHomePage/diaspora/
1,AFCON,https://www.ghanaweb.com/afcon/
2,African News,https://www.ghanaweb.com/GhanaHomePage/africa/
3,"Articles, feature",https://www.ghanaweb.com/GhanaHomePage/feature...
4,Athletes,https://www.ghanaweb.com/GhanaHomePage/people/...


## Selecting Story Sections with articles

In [27]:
section_list = ["Abroad, Ghanaians","AFCON","African News","Athletics","BBC Hausa News", "BBC Pidgin News","Business & Economy",
"Coronavirus","Crime News","Editorial News","Entertainment","Health News","Lifestyle","Music","Political News",
"Regional News","Sports Section","Tabloid News"]

In [28]:
def get_section_urls(row):
    df = pd.DataFrame()
    if row['Section'] in section_list:
        temp_row = pd.DataFrame(data = [[row["Section"], row["url"]]], columns = df_columns)
        df = pd.concat([df,temp_row],ignore_index =True)
    else:
        pass

    return df

In [29]:
df = sections_df.apply(lambda row : get_section_urls(row),axis = 1)

In [30]:
df_columns = ['Section','url']
stories_urls_df = pd.DataFrame(columns = df_columns)

In [31]:
for row in df:
    stories_urls_df = stories_urls_df.append(row)

In [32]:
stories_urls_df.reset_index(inplace=True)

In [33]:
del stories_urls_df['index']
stories_urls_df.head()

,Section,url
0,"Abroad, Ghanaians",https://www.ghanaweb.com/GhanaHomePage/diaspora/
1,AFCON,https://www.ghanaweb.com/afcon/
2,African News,https://www.ghanaweb.com/GhanaHomePage/africa/
3,Athletics,https://www.ghanaweb.com/GhanaHomePage/athletics/
4,BBC Hausa News,https://www.ghanaweb.com/GhanaHomePage/bbc/hausa


In [34]:
stories_urls_df.to_csv("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\CSV files\\Selected_Sections_data.csv")